In [ ]:
from google.colab import files
import os
import zipfile
import pandas as pd
from tqdm.notebook import tqdm
import re
import random
import collections
from sklearn.model_selection import train_test_split

from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

files.upload()

if not os.path.exists("/root/.kaggle/kaggle.json"):
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("Kaggle API key đã được thiết lập.")
else:
    print("Kaggle API key đã tồn tại.")

In [ ]:
dataset_slug = 'cngonngc/phomt-datasetvinai'
zip_file_name = dataset_slug.split('/')[-1] + '.zip'
extract_path = './phomt_dataset_vinai'

!kaggle datasets download -d {dataset_slug}

if not os.path.exists(zip_file_name):
    raise FileNotFoundError(f"File zip '{zip_file_name}' không được tải xuống thành công. Vui lòng kiểm tra lại lệnh download và kết nối mạng.")

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
base_data_path = extract_path + '/'

train_en_path = os.path.join(base_data_path, 'train.en')
train_vi_path = os.path.join(base_data_path, 'train.vi')

dev_en_path = os.path.join(base_data_path, 'dev.en')
dev_vi_path = os.path.join(base_data_path, 'dev.vi')

test_en_path = os.path.join(base_data_path, 'test.en')
test_vi_path = os.path.join(base_data_path, 'test.vi')

def read_parallel_data(en_path, vi_path):
    if not os.path.exists(en_path):
        raise FileNotFoundError(f"Lỗi: Không tìm thấy file tiếng Anh: {en_path}")
    if not os.path.exists(vi_path):
        raise FileNotFoundError(f"Lỗi: Không tìm thấy file tiếng Việt: {vi_path}")

    with open(en_path, 'r', encoding='utf-8') as f_en, \
            open(vi_path, 'r', encoding='utf-8') as f_vi:
        en_sentences = f_en.readlines()
        vi_sentences = f_vi.readlines()

    if len(en_sentences) != len(vi_sentences):
        min_len = min(len(en_sentences), len(vi_sentences))
        en_sentences = en_sentences[:min_len]
        vi_sentences = vi_sentences[:min_len]

    return pd.DataFrame({'English': en_sentences, 'Vietnamese': vi_sentences})

df_train = read_parallel_data(train_en_path, train_vi_path)
df_dev = read_parallel_data(dev_en_path, dev_vi_path)
df_test = read_parallel_data(test_en_path, test_vi_path)

print(f"Số cặp câu huấn luyện gốc: {len(df_train)}")
print(f"Số cặp câu phát triển gốc: {len(df_dev)}")
print(f"Số cặp câu kiểm thử gốc: {len(df_test)}")

print("\n5 dòng đầu của dữ liệu:")
print(df_train.head())

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = re.sub(r'[^\w\s.,!?;:\-–—"\'“”‘’()\[\]{}<>áàảãạăằẳẵặâầẩẫậéèẻẽẹêềểễệíìỉĩịóòỏõọôồổỗộơờởỡợúùủũụưừửữựýỳỷỹỵđ]', '', text)
    return text


# Làm sạch dữ liệu

In [ ]:
df_train_cleaned = df_train.copy()
df_dev_cleaned = df_dev.copy()
df_test_cleaned = df_test.copy()

for col in ['English', 'Vietnamese']:
    df_train_cleaned[col] = df_train_cleaned[col].str.replace('\n', '', regex=False)
    df_dev_cleaned[col] = df_dev_cleaned[col].str.replace('\n', '', regex=False)
    df_test_cleaned[col] = df_test_cleaned[col].str.replace('\n', '', regex=False)

    df_train_cleaned[col] = df_train_cleaned[col].apply(clean_text)
    df_dev_cleaned[col] = df_dev_cleaned[col].apply(clean_text)
    df_test_cleaned[col] = df_test_cleaned[col].apply(clean_text)

min_sentence_length = 3
initial_train_len = len(df_train_cleaned)
df_train_cleaned = df_train_cleaned[
    (df_train_cleaned['English'].str.len() >= min_sentence_length) &
    (df_train_cleaned['Vietnamese'].str.len() >= min_sentence_length)
]
df_dev_cleaned = df_dev_cleaned[
    (df_dev_cleaned['English'].str.len() >= min_sentence_length) &
    (df_dev_cleaned['Vietnamese'].str.len() >= min_sentence_length)
]
df_test_cleaned = df_test_cleaned[
    (df_test_cleaned['English'].str.len() >= min_sentence_length) &
    (df_test_cleaned['Vietnamese'].str.len() >= min_sentence_length)
]

df_train_cleaned.drop_duplicates(subset=['English', 'Vietnamese'], inplace=True)
df_dev_cleaned.drop_duplicates(subset=['English', 'Vietnamese'], inplace=True)
df_test_cleaned.drop_duplicates(subset=['English', 'Vietnamese'], inplace=True)

print(f"Số cặp câu huấn luyện sau làm sạch ban đầu: {len(df_train_cleaned)} (Đã loại bỏ {initial_train_len - len(df_train_cleaned)} cặp)")
print(f"Số cặp câu phát triển sau làm sạch ban đầu: {len(df_dev_cleaned)}")
print(f"Số cặp câu kiểm thử sau làm sạch ban đầu: {len(df_test_cleaned)}")